# Notes
WSZYSTKIE PLIKI SĄ IDENTYCZNE. WSZYSTKIE TO EXCEL - 14 kart
Czyszczenie:
•	Wszystkie NaN na 0
•	Skalowanie przed modelem
•	Inżynieria cech (wskaźniki ekonomiczne, różnica miedzy każym z elementów)

TESTY DO PRZEPROWADZENIA:
•	Uwzglednienie wszystkiego
•	Skupienie sie na zmianach kapitałowych
•	Ograniczenie tylko główne wartości
•	Grupowanie po przedsiębiorstwie? 

Podsumowanie co wyciągnąć z pliku:
Dodatkowa kolumna jako nazwa pliku
C – nazwa kolumny w DF


In [235]:
import pandas as pd
import re
from data_utils import *

In [236]:
names_data = pd.read_csv('dataframes/names_gpw.csv')

In [237]:
def clean_column_names(columns):
    cleaned_columns = []
    for column in columns:
        column = column.lower()
        column = re.sub(r'\(.*?\)', '', column)
        column = re.sub(r'[^\w\s]', '', column)
        column = re.sub(r'\s+', '_', column)
        column = column.rstrip('_')
        cleaned_columns.append(column)
    return cleaned_columns

In [238]:
def count_rows_by_filename(df):
    counts = df.groupby('filename').size().reset_index(name='count')
    return counts

# FULL DF CLEARNING

In [239]:
full_data = pd.read_csv("dataframes/dataframe_gpw.csv")

In [240]:
data_gpw = pd.merge(full_data, names_data, on='filename')

In [241]:
data_gpw = data_gpw.sort_values(by=['filename', 'End of period'])

In [242]:
data_gpw.columns = clean_column_names(data_gpw.columns)

In [243]:
count_rows_by_filename(data_gpw)

,filename,count
0,11BIT.xlsx,49
1,3LPSA.xlsx,6
2,3RGAMES.xlsx,48
3,AB.xlsx,70
4,AC.xlsx,51
...,...,...
332,ZPUE.xlsx,95
333,ZREMBCH.xlsx,65
334,ZUE.xlsx,53
335,ZUK.xlsx,67


In [244]:
# Delete all companies under 4 because its not enough to cover any change in assets or liabilities
filenames_to_remove = [
    'SADOVAYA.xlsx',
    'BMW.xlsx',
    'KERNEL.xlsx',
    'SILVANO.xlsx',
    'IMCSA.xlsx',
    'STSHOLDING.xlsx'
]

df_filtered = data_gpw[~data_gpw['filename'].isin(filenames_to_remove)]

# Missing data

In [245]:
data_gpw.isnull().sum()

end_of_period                                                      0
revenues_from_sales                                              945
operating_profitloss                                             737
net_profitloss_attributable_to_equity_holders_of_the_parent      577
total_comprehensive_income_attributable_to_equity_holders      17723
                                                               ...  
noncurrent_financial_liabilities                               19906
financial_liabilities                                          19906
company_name                                                       0
sector                                                            13
ticker                                                             0
Length: 109, dtype: int64

In [246]:
# Sektor NaN
rows_with_nan_in_sektor = data_gpw[data_gpw['sector'].isna()]
rows_with_nan_in_sektor

,end_of_period,revenues_from_sales,operating_profitloss,net_profitloss_attributable_to_equity_holders_of_the_parent,total_comprehensive_income_attributable_to_equity_holders,depreciation,cash_flow_from_operating_activities,cash_flow_from_investing_activities,cash_flow_from_financing_activities,aktywa,...,rate_debt_security,leverage,asset_utilization,load_gross_profit,current_financial_liabilities,noncurrent_financial_liabilities,financial_liabilities,company_name,sector,ticker
49,2017-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.301800e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3LP SA,NaN,3LP_IPO
50,2018-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.053760e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3LP SA,NaN,3LP_IPO
51,2019-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.613490e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3LP SA,NaN,3LP_IPO
52,2020-12-31,34515.00000,3929.00000,138.00000,138.0,4114.00000,8513.00000,-3261.00000,-5264.0000,1.640630e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3LP SA,NaN,3LP_IPO
53,2021-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.547040e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3LP SA,NaN,3LP_IPO
54,2021-12-31,39704.00000,3898.00000,2169.00000,2169.0,4459.00000,5831.00000,-2718.00000,-3195.0000,1.571200e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3LP SA,NaN,3LP_IPO
2522,2020-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.166580e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BMW AG,NaN,BMW
2523,2021-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.295270e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BMW AG,NaN,BMW
18362,2019-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.065330e+03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Uf Games SA,NaN,UFG_IPO
18363,2020-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.348397e+03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Uf Games SA,NaN,UFG_IPO


## Notes
Manually checking for each company

3LP SA - handel
BMW AG - motoryzacja
Uf Games SA - gry


In [247]:
company_to_sector = {
    '3LP SA': 'handel',
    'BMW AG': 'motoryzacja',
    'Uf Games SA': 'gry'
}

data_gpw['sector'] = data_gpw.apply(lambda row: company_to_sector[row['company_name']] if pd.isna(row['sector']) else row['sector'], axis=1)

In [248]:
# Rest of financial data - fill NaN with 0 (=no change)
data_gpw = data_gpw.fillna(0)

In [249]:
data_gpw.columns.tolist()

['end_of_period',
 'revenues_from_sales',
 'operating_profitloss',
 'net_profitloss_attributable_to_equity_holders_of_the_parent',
 'total_comprehensive_income_attributable_to_equity_holders',
 'depreciation',
 'cash_flow_from_operating_activities',
 'cash_flow_from_investing_activities',
 'cash_flow_from_financing_activities',
 'aktywa',
 'noncurrent_assets',
 'current_assets',
 'equity_shareholders_of_the_parent',
 'noncurrent_liabilities',
 'current_liabilities',
 'number_of_shares',
 'earnings_per_share',
 'total_comprehensive_income_per_share',
 'book_value_per_share',
 'accounting_standard',
 'filename',
 'property_plant_and_equipment',
 'exploration_for_and_evaluation_of_mineral_resources',
 'intangible_assets',
 'goodwill',
 'investment_property',
 'rightofuse_assets',
 'investment_in_affiliates',
 'noncurrent_financial_assets',
 'noncurrent_loans_and_receivables',
 'deferred_income_tax',
 'noncurrent_deferred_charges_and_accruals',
 'noncurrent_derivative_instruments',
 'other

## Change each indicator to % value
return_on_equity
return_on_assets
leverage
asset_utilization
debt_ratio
debt_service_ratio
rate_debt_security
working_capital_ratio

In [250]:
columns_to_convert = [
    'return_on_equity',
    'return_on_assets',
    'leverage',
    'asset_utilization',
    'debt_ratio',
    'debt_service_ratio',
    'rate_debt_security',
    'working_capital_ratio'
]

data_gpw[columns_to_convert] = data_gpw[columns_to_convert] / 100

In [251]:
column_mapping = {
    'financial_liabilities': 'total_liability',
    'aktywa': 'total_assets'
}

data_gpw = data_gpw.rename(columns=column_mapping)

In [252]:
data_gpw['company_name'].unique()

array(['11 bit studios SA', '3LP SA', '3R Games SA', 'AB SA', 'AC SA',
       'Action SA', 'Adiuvo Investments SA', 'Agora SA', 'Agroton plc.',
       'AUGA Group AB', 'All in! Games SA', 'Ailleron SA',
       'Airway Medix SA', 'Alumetal SA', 'Ambra SA', 'Amica SA',
       'Ampli SA w upadłości układowej', 'AmRest Holdings SE',
       'Answear.com SA', 'Apator SA', 'Aplisens SA', 'APS Energia SA',
       'Arctic Paper SA', 'Arteria SA', 'Artifex Mundi SA',
       'ASBISc Enterprises plc.', 'ASM Group SA', 'Asseco BS SA',
       'Asseco Poland SA', 'Asseco SEE SA', 'Atende SA',
       'Atlanta Poland SA', 'ATM Grupa SA', 'Atrem SA', 'Auto Partner SA',
       'Grupa Azoty SA', 'British Automotive Holding SA',
       'Elektrociepłownia Będzin SA', 'Benefit Systems SA', 'Berling SA',
       'Betacom SA', 'Big Cheese Studio SA', 'Bioceltix SA',
       'BioMaxima SA', 'Biomed Lublin SA', 'Bio Planet SA', 'Bioton SA',
       'BMW AG', 'LW Bogdanka SA', 'BoomBit SA', 'Boryszew SA',
       'Bo

# Feature Engineering

## Ticket for yahoo finance API

In [253]:
data_gpw

,end_of_period,revenues_from_sales,operating_profitloss,net_profitloss_attributable_to_equity_holders_of_the_parent,total_comprehensive_income_attributable_to_equity_holders,depreciation,cash_flow_from_operating_activities,cash_flow_from_investing_activities,cash_flow_from_financing_activities,total_assets,...,rate_debt_security,leverage,asset_utilization,load_gross_profit,current_financial_liabilities,noncurrent_financial_liabilities,total_liability,company_name,sector,ticker
0,2010-03-31,66.0,0.0,-16.0,0.0,29.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11 bit studios SA,gry,11B
1,2010-09-30,129.0,0.0,-26.0,0.0,6.0,0.0,0.0,0.0,1411.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11 bit studios SA,gry,11B
2,2010-12-31,228.0,0.0,-198.0,0.0,0.0,0.0,0.0,0.0,1352.36,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11 bit studios SA,gry,11B
3,2011-03-31,144.0,0.0,-50.0,0.0,0.0,0.0,0.0,0.0,1342.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11 bit studios SA,gry,11B
4,2011-06-30,959.0,0.0,377.0,0.0,7.0,0.0,0.0,0.0,1659.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11 bit studios SA,gry,11B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19901,2021-09-30,995385.0,103668.0,104744.0,106250.0,36985.0,354390.0,-22037.0,-433167.0,2649304.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Grupa Żywiec SA,napoje,ZWC
19902,2021-12-31,733974.0,46890.0,32175.0,42434.0,42311.0,50929.0,-65863.0,30091.0,2508447.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Grupa Żywiec SA,napoje,ZWC
19903,2022-03-31,696778.0,6649.0,-1246.0,41367.0,38391.0,13967.0,-33280.0,57099.0,2741947.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Grupa Żywiec SA,napoje,ZWC
19904,2022-06-30,1209418.0,101579.0,92625.0,45386.0,39932.0,101877.0,-21179.0,-81990.0,2981239.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Grupa Żywiec SA,napoje,ZWC


In [254]:
def add_suffix_to_ticker(df):
    df['ticker'] = df['ticker'].astype(str) + '.WA'
    return df

df = add_suffix_to_ticker(data_gpw)

In [255]:
failed_tickers = [] #do zapisanie tych do nie przechodzą

def fetch_market_value(row):
    market_value = get_market_value(row['ticker'], row['end_of_period'])
    if market_value is None:
        failed_tickers.append(row['ticker'])
    return market_value

# df['market_value'] = df.apply(fetch_market_value, axis=1) - failed (no data in yahoo finance)

## Market Value directly from files

In [256]:
market_value = pd.read_csv('marketvalue_data/company_market_value.csv')

In [257]:
market_df = pd.DataFrame(market_value)

In [258]:
market_df

,Name,Market value,Period end
0,06MAGNA,2.784000e+07,2013-12-31
1,06MAGNA,2.951000e+07,2014-12-31
2,06MAGNA,2.534000e+07,2015-12-31
3,06MAGNA,1.058000e+07,2016-12-31
4,06MAGNA,5.150000e+06,2017-12-31
...,...,...,...
4140,ZYWIEC,4.848070e+09,2017-12-31
4141,ZYWIEC,4.745360e+09,2018-12-31
4142,ZYWIEC,5.032960e+09,2019-12-31
4143,ZYWIEC,4.971330e+09,2020-12-31


In [259]:
def add_suffix_to_market(df):
    df['Name'] = df['Name'].astype(str) + '.xlsx'
    df['filename'] = df['Name']
    df['end_of_period'] = df['Period end']
    return df

market_df = add_suffix_to_market(market_df)

In [260]:
if 'filename' not in df.columns or 'end_of_period' not in df.columns:
    raise ValueError("Brakuje kolumn 'company' lub 'end_of_period' w załadowanym DataFrame")

merged_df = pd.merge(df, market_df, on=["filename", "end_of_period"], how="left")

In [261]:
merged_df

,end_of_period,revenues_from_sales,operating_profitloss,net_profitloss_attributable_to_equity_holders_of_the_parent,total_comprehensive_income_attributable_to_equity_holders,depreciation,cash_flow_from_operating_activities,cash_flow_from_investing_activities,cash_flow_from_financing_activities,total_assets,...,load_gross_profit,current_financial_liabilities,noncurrent_financial_liabilities,total_liability,company_name,sector,ticker,Name,Market value,Period end
0,2010-03-31,66.0,0.0,-16.0,0.0,29.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,11 bit studios SA,gry,11B.WA,NaN,NaN,NaN
1,2010-09-30,129.0,0.0,-26.0,0.0,6.0,0.0,0.0,0.0,1411.00,...,0.0,0.0,0.0,0.0,11 bit studios SA,gry,11B.WA,NaN,NaN,NaN
2,2010-12-31,228.0,0.0,-198.0,0.0,0.0,0.0,0.0,0.0,1352.36,...,0.0,0.0,0.0,0.0,11 bit studios SA,gry,11B.WA,NaN,NaN,NaN
3,2011-03-31,144.0,0.0,-50.0,0.0,0.0,0.0,0.0,0.0,1342.00,...,0.0,0.0,0.0,0.0,11 bit studios SA,gry,11B.WA,NaN,NaN,NaN
4,2011-06-30,959.0,0.0,377.0,0.0,7.0,0.0,0.0,0.0,1659.00,...,0.0,0.0,0.0,0.0,11 bit studios SA,gry,11B.WA,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19901,2021-09-30,995385.0,103668.0,104744.0,106250.0,36985.0,354390.0,-22037.0,-433167.0,2649304.00,...,0.0,0.0,0.0,0.0,Grupa Żywiec SA,napoje,ZWC.WA,NaN,NaN,NaN
19902,2021-12-31,733974.0,46890.0,32175.0,42434.0,42311.0,50929.0,-65863.0,30091.0,2508447.00,...,0.0,0.0,0.0,0.0,Grupa Żywiec SA,napoje,ZWC.WA,ZYWIEC.xlsx,5.197300e+09,2021-12-31
19903,2022-03-31,696778.0,6649.0,-1246.0,41367.0,38391.0,13967.0,-33280.0,57099.0,2741947.00,...,0.0,0.0,0.0,0.0,Grupa Żywiec SA,napoje,ZWC.WA,NaN,NaN,NaN
19904,2022-06-30,1209418.0,101579.0,92625.0,45386.0,39932.0,101877.0,-21179.0,-81990.0,2981239.00,...,0.0,0.0,0.0,0.0,Grupa Żywiec SA,napoje,ZWC.WA,NaN,NaN,NaN


In [262]:
# Removing everything without target

# Sprawdź, czy kolumna 'Market value' istnieje w załadowanym DataFrame
if 'Market value' not in merged_df.columns:
    raise ValueError("Kolumna 'Market value' nie istnieje w załadowanym DataFrame")

# Usuń wiersze z NaN w kolumnie 'Market value'
filtered_df = merged_df.dropna(subset=['Market value'])

# Usuń wiersze z 0 w kolumnie 'Market value'
filtered_df = filtered_df[filtered_df['Market value'] != 0]

In [263]:
filtered_df

,end_of_period,revenues_from_sales,operating_profitloss,net_profitloss_attributable_to_equity_holders_of_the_parent,total_comprehensive_income_attributable_to_equity_holders,depreciation,cash_flow_from_operating_activities,cash_flow_from_investing_activities,cash_flow_from_financing_activities,total_assets,...,load_gross_profit,current_financial_liabilities,noncurrent_financial_liabilities,total_liability,company_name,sector,ticker,Name,Market value,Period end
22,2015-12-31,7118.055,3766.639,3131.438,0.000,479.143,3600.039,-161.750,386.591,33895.516,...,0.0,0.0,0.0,0.0,11 bit studios SA,gry,11B.WA,11BIT.xlsx,1.565300e+08,2015-12-31
26,2016-12-31,9255.686,5223.702,3752.445,0.000,635.822,5357.262,-3043.810,-305.956,45350.347,...,0.0,0.0,0.0,0.0,11 bit studios SA,gry,11B.WA,11BIT.xlsx,3.254800e+08,2016-12-31
30,2017-12-31,7749.556,5271.914,3976.657,0.000,367.638,4117.506,-3352.652,0.000,48649.817,...,0.0,0.0,0.0,0.0,11 bit studios SA,gry,11B.WA,11BIT.xlsx,4.528700e+08,2017-12-31
34,2018-12-31,22493.025,11724.432,9031.812,37549.897,1590.751,4319.939,-22251.450,12600.000,109262.821,...,0.0,0.0,0.0,0.0,11 bit studios SA,gry,11B.WA,11BIT.xlsx,5.580800e+08,2018-12-31
38,2019-12-31,21791.225,14140.468,15218.158,15218.158,1843.751,5013.209,-3565.279,-414.282,155668.388,...,0.0,0.0,0.0,0.0,11 bit studios SA,gry,11B.WA,11BIT.xlsx,9.148800e+08,2019-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19886,2017-12-31,640925.000,40794.000,31137.000,0.000,28418.000,69027.000,-38611.000,-16068.000,1778284.000,...,0.0,0.0,0.0,0.0,Grupa Żywiec SA,napoje,ZWC.WA,ZYWIEC.xlsx,4.848070e+09,2017-12-31
19890,2018-12-31,725450.000,91325.000,70684.000,0.000,28672.000,20399.000,-52414.000,-1149.000,1790063.000,...,0.0,0.0,0.0,0.0,Grupa Żywiec SA,napoje,ZWC.WA,ZYWIEC.xlsx,4.745360e+09,2018-12-31
19894,2019-12-31,651502.000,40236.000,30791.000,335501.000,31722.000,145523.000,-78807.000,-13101.000,2479535.000,...,0.0,0.0,0.0,0.0,Grupa Żywiec SA,napoje,ZWC.WA,ZYWIEC.xlsx,5.032960e+09,2019-12-31
19898,2020-12-31,770024.000,96982.000,69584.000,75207.000,37321.000,6740.000,85060.000,-18936.000,2513606.000,...,0.0,0.0,0.0,0.0,Grupa Żywiec SA,napoje,ZWC.WA,ZYWIEC.xlsx,4.971330e+09,2020-12-31


In [265]:
filtered_df.to_csv("dataframes/data_with_target.csv", index=False)